### Step-by-step transformation pipeline for validation + vehicle events + calendar join

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Optional
import sys, os
sys.path.append(os.path.abspath(".."))

In [3]:
validations_path: Optional[str] = "data/apex-validations/apex-validations.csv"
events_path: Optional[str] = "data/vehicle-events/vehicle-events.csv" 
calendar_path: Optional[str] = "data/calendario.xlsx"
stops_path: Optional[str] = "data/stops.csv"

### Helper functions

In [4]:
VALID_STATUSES = {0, 4, 5, 6}

def read_any(path: str) -> pd.DataFrame:
    p = Path(path)
    if p.suffix.lower() == ".xlsx":
        return pd.read_excel(p)
    elif p.suffix.lower() in {".csv", ".txt"}:
        return pd.read_csv(p)
    elif p.suffix.lower() in {".json"}:
        return pd.read_json(p, lines=True)
    else:
        raise ValueError(f"Unsupported file type: {p.suffix}")

def to_datetime_from_ms(series: pd.Series) -> pd.Series:
    # Handles ints/floats/strings like "1.75129E+12"
    # Ensure numeric milliseconds, then convert to UTC datetime
    ser = pd.to_numeric(series, errors="coerce")
    return pd.to_datetime(ser, unit="ms", utc=True)

# Convert booleans and ints where appropriate
def to_int(series):
    return pd.to_numeric(series, errors="coerce").astype("Int64")

def to_float(series):
    return pd.to_numeric(series, errors="coerce")


### 1. Load Data

In [5]:
if validations_path:
    df_valid = read_any(validations_path)

if events_path:
    df_event = read_any(events_path)

if calendar_path:
    df_cal = read_any(calendar_path)

if stops_path:
    stops = read_any(stops_path)

/var/folders/hh/pvwk6s4917qdt0_j791cx6wc0000gn/T/ipykernel_2707/1083672054.py:8: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(p)
/var/folders/hh/pvwk6s4917qdt0_j791cx6wc0000gn/T/ipykernel_2707/1083672054.py:8: DtypeWarning: Columns (4,13) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(p)
/var/folders/hh/pvwk6s4917qdt0_j791cx6wc0000gn/T/ipykernel_2707/1083672054.py:8: DtypeWarning: Columns (50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(p)


### 2. Clean + Timestamp conversion

In [6]:
for col in ("created_at","received_at"):
    if col in df_valid.columns:
        df_valid[col] = to_datetime_from_ms(df_valid[col])
        df_event[col] = to_datetime_from_ms(df_event[col])

### 3. Normalize specific fields

In [7]:
# Filter: passenger validations & valid statuses
if "is_passenger" in df_valid.columns:
    df_valid = df_valid[df_valid["is_passenger"].astype(bool)]
if "validation_status" in df_valid.columns:
    df_valid = df_valid[df_valid["validation_status"].isin(VALID_STATUSES)]

In [7]:
df_valid["vehicle_id"] = to_int(df_valid["vehicle_id"])
df_valid["stop_id"] = to_int(df_valid["stop_id"])

In [8]:
stops['stop_id'] = to_int(stops["stop_id"])
stops["stop_lat"] = to_float(stops["stop_lat"])
stops["stop_lon"] = to_float(stops["stop_lon"])

In [9]:
df_event["vehicle_id"] = to_int(df_event["vehicle_id"])
df_event["stop_id"] = to_int(df_event["stop_id"])
df_event["latitude"] = to_float(df_event["latitude"])
df_event["longitude"] = to_float(df_event["longitude"])

### 4. Calendar join

In [10]:
df_valid["service_date"] = df_valid["created_at"].dt.tz_convert("UTC").dt.date
df_valid["date_int"] = (df_valid["service_date"].astype(str).str.replace("-","", regex=False)).astype(int)

In [11]:
df_event["service_date"] = df_event["created_at"].dt.tz_convert("UTC").dt.date
df_event["date_int"] = (df_event["service_date"].astype(str).str.replace("-","", regex=False)).astype(int)

In [12]:
df_cal = df_cal.rename(columns={"date":"date_int"})
# Ensure date_int is integer
df_cal["date_int"] = pd.to_numeric(df_cal["date_int"], errors="coerce").astype("Int64")

In [13]:
df_valid = df_valid.merge(df_cal, on="date_int", how="left")

### 5. Aggregations

#### A) Passengers per stop_id per day (with day_type/dia_tipo)

In [15]:
per_stop_day = (
    df_valid
    .dropna(subset=["stop_id"])
    .groupby(["stop_id","service_date","day_type","dia_tipo"], dropna=False)
    .size()
    .reset_index(name="num_passengers")
    .sort_values(["service_date","stop_id"])
)

In [17]:
per_stop_day

,stop_id,service_date,day_type,dia_tipo,num_passengers
7,10009,2025-10-13,1,DU,561
15,10010,2025-10-13,1,DU,398
22,10011,2025-10-13,1,DU,1
23,10012,2025-10-13,1,DU,5
29,10018,2025-10-13,1,DU,1
...,...,...,...,...,...
61439,180709,2025-10-20,1,DU,4
61508,180723,2025-10-20,1,DU,4
61523,180727,2025-10-20,1,DU,1
61531,180730,2025-10-20,1,DU,6


#### B) Total passengers per stop_id (all days)

In [201]:
per_stop_total = (
    per_stop_day
    .groupby("stop_id", as_index=False)["num_passengers"]
    .sum()
    .rename(columns={"num_passengers":"num_passengers_total"})
    .sort_values("num_passengers_total", ascending=False)
)

In [202]:
per_stop_total

,stop_id,num_passengers_total
934,30119,23143
1963,60002,16761
1962,60001,16751
1995,60071,15771
5193,111025,15173
...,...,...
3203,71493,1
6207,130247,1
3216,71515,1
6204,130244,1


### 6. Aggregating datasets

In [14]:
agg_validations = (
    df_valid.groupby(["stop_id", "trip_id", "vehicle_id", "dia_tipo", "line_id", "pattern_id", "service_date"], as_index=False)
      .agg(
          num_validations=("card_serial_number", "count"),        # total validations
          num_unique_cards=("card_serial_number", "nunique"),     # unique passengers/cards
          created_at=("created_at", "first"), 
          day_type=("day_type", "first"),
          day_tipo=("dia_tipo", "first"),
          service_date=("service_date", "first"),
          date_int=("date_int", "first")
      )
      .sort_values(["vehicle_id", "stop_id"])
)

In [ ]:
#agg = (
 #   df_event.groupby(["stop_id", "trip_id", "pattern_id", "vehicle_id"], as_index=False)
 #     .agg(
 #         open_flag=("trigger_door", lambda x: int((x == "OPENED").any())),
 #         nochange_flag=("trigger_door", lambda x: int((x == "NO_CHANGE").any())),
 #         latitude=("latitude", "first"),
  #        longitude=("longitude", "first"),
  #        created_at=("created_at", "first"),
  #        received_at=("received_at", "last"),
  #        date_int=("date_int", "first"),
  #        driver_id=("vehicle_id", "first")
  #    )
#)

# optional: sort or re-order
#agg = agg.sort_values(["vehicle_id", "trip_id", "stop_id"]).reset_index(drop=True)


In [204]:
agg_validations.loc[(agg_validations["trip_id"]=="1709_0_1_0500_0529_0_1") & (agg_validations['stop_id'] == 110027)]

,stop_id,trip_id,vehicle_id,dia_tipo,line_id,pattern_id,num_validations,num_unique_cards,created_at,day_type,day_tipo,service_date,date_int
573739,110027,1709_0_1_0500_0529_0_1,1203,DU,1709,1709_0_1,12,12,2025-10-13 04:00:03+00:00,1,DU,2025-10-13,20251013
573740,110027,1709_0_1_0500_0529_0_1,1203,DU,1709,1709_0_1,2,2,2025-10-14 03:59:17+00:00,1,DU,2025-10-14,20251014
573741,110027,1709_0_1_0500_0529_0_1,1203,DU,1709,1709_0_1,1,1,2025-10-15 03:59:21+00:00,1,DU,2025-10-15,20251015
573742,110027,1709_0_1_0500_0529_0_1,1203,DU,1709,1709_0_1,2,2,2025-10-16 04:00:18+00:00,1,DU,2025-10-16,20251016
573743,110027,1709_0_1_0500_0529_0_1,1203,DU,1709,1709_0_1,3,3,2025-10-17 03:59:31+00:00,1,DU,2025-10-17,20251017


In [26]:
stops

,stop_id,stop_name,stop_name_new,stop_short_name,stop_lat,stop_lon,operational_status,areas,region_id,region_name,...,near_transit_office,near_beach,subway,light_rail,train,boat,airport,bike_sharing,bike_parking,car_parking
0,10001,Rua Carlos Manuel Rodrigues Francisco (Escola),Rua Carlos Manuel Rodrigues Francisco (Escola),R. Carlos Manuel Rodrigues Francisco (Escola),38.754244,-8.959557,ACTIVE,44,PT170,AML,...,1,0,0,0,0,0,0,0,0,0
1,10002,R Carlos M. Francisco 229 (Escola Monte Novo),Rua Carlos Manuel Rodrigues Francisco (Frente ...,R. Carlos Manuel Rodrigues Francisco (Ft. Escola),38.754572,-8.959615,ACTIVE,44,PT170,AML,...,1,0,0,0,0,0,0,0,0,0
2,10005,ALCOCHETE (R CIPRIÃO FIGUEIREDO),Rua Doutor Ciprião Figueiredo 13,R Dr Ciprião Figueiredo 13,38.754175,-8.961806,ACTIVE,44,PT170,AML,...,0,0,0,0,0,0,0,0,0,0
3,10007,ALCOCHETE (R LEITE CUNHA) BIBLIOTECA,Rua Professor Leite Cunha (Biblioteca),R. Prof. Leite Cunha (Biblioteca),38.753196,-8.963687,ACTIVE,44,PT170,AML,...,0,0,0,0,0,0,0,0,0,0
4,10008,ALCOCHETE (R LEITE CUNHA) BIBLIOTECA,Rua Professor Leite Cunha (Frente Biblioteca),R. Prof. Leite Cunha (Ft. Biblioteca),38.753271,-8.963504,ACTIVE,44,PT170,AML,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12680,208398,Estrada das Corredouras 37,Estrada das Corredouras 37,ESTR DAS CORREDOURAS 37,38.995096,-9.073490,ACTIVE,42,PT16B,Oeste,...,0,0,0,0,0,0,0,0,0,0
12681,208456,Estrada das Corredouras (X) Zona Indústrial,Estrada das Corredouras (X) Zona Indústrial,ESTR DAS CORREDOURAS (X) ZN INDÚSTRIAL,39.000590,-9.066633,ACTIVE,42,PT16B,Oeste,...,0,0,0,0,0,0,0,0,0,0
12682,209650,Estrada das Corredouras (Ft Zona Industrial),Estrada das Corredouras (Ft Zona Industrial),ESTR DAS CORREDOURAS (FT ZN INDUSTRIAL),39.000470,-9.066970,ACTIVE,42,PT16B,Oeste,...,0,0,0,0,0,0,0,0,0,0
12683,999991,Aiana (Cemitério),Aiana (Cemitério),Aiana (Cemitério),38.492743,-9.127095,ACTIVE,NaN,PT170,AML,...,0,0,0,0,0,0,0,0,0,0


In [63]:
events = df_event.sort_values(["vehicle_id", "stop_id", "created_at"]).copy()

In [67]:
events

,_id,agency_id,created_at,driver_id,extra_trip_id,latitude,longitude,odometer,pattern_id,received_at,stop_id,trigger_door,trip_id,vehicle_id,service_date,date_int
24179964,68f267fcabdd6324a4d45d29,43,2025-10-17 15:58:37+00:00,FFP18G,NaN,38.650478,-9.152249,192633808,NaN,2025-10-17 15:59:56.936000+00:00,1,NO_CHANGE,NaN,14,2025-10-17,20251017
24188019,68f2684be2dc246c99b9d9a5,43,2025-10-17 16:01:14+00:00,FFP18G,NaN,38.650208,-9.152725,192634048,NaN,2025-10-17 16:01:15.551000+00:00,1,OPENED,NaN,14,2025-10-17,20251017
24188513,68f26854df3e447bc9322fff,43,2025-10-17 16:01:23+00:00,FFP18G,NaN,38.650208,-9.152727,192634048,NaN,2025-10-17 16:01:24.665000+00:00,1,CLOSED,NaN,14,2025-10-17,20251017
220343,68ec949d50d52e23e1e3b42e,43,2025-10-13 05:56:44+00:00,ZBJHNF,NaN,38.673817,-9.153441,192000944,NaN,2025-10-13 05:56:45.788000+00:00,2,NO_CHANGE,NaN,14,2025-10-13,20251013
1215100,68ecc2443bad5529d99c3f23,43,2025-10-13 09:11:30+00:00,ZBJHNF,NaN,38.687992,-9.147430,192044752,NaN,2025-10-13 09:11:31.790000+00:00,2,NO_CHANGE,NaN,14,2025-10-13,20251013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7462865,68ee49937bdfa272f2964571,41,2025-10-14 13:01:06+00:00,5JE2NI,1001_0_1_20251014T130037_338f7cee,0.000000,0.000000,-1,1001_0_1,2025-10-14 13:01:07.965000+00:00,<NA>,NO_CHANGE,1001_0_1_20251014T130037_338f7cee,<NA>,2025-10-14,20251014
7574522,68ee4eba1d39da2af2dcc93a,41,2025-10-14 13:23:05+00:00,5JE2NI,1001_0_1_20251014T131834_a9eee395,0.000000,0.000000,-1,1001_0_1,2025-10-14 13:23:05.994000+00:00,<NA>,NO_CHANGE,1001_0_1_20251014T131834_a9eee395,<NA>,2025-10-14,20251014
7618721,68ee50d8abdd6324a4bb3646,41,2025-10-14 13:32:07+00:00,5JE2NI,1001_0_1_20251014T133055_3bc75041,0.000000,0.000000,-1,1001_0_1,2025-10-14 13:32:08.553000+00:00,<NA>,NO_CHANGE,1001_0_1_20251014T133055_3bc75041,<NA>,2025-10-14,20251014
7644615,68ee52157bdfa272f2968d70,41,2025-10-14 13:37:24+00:00,5JE2NI,1001_0_2_20251014T133723_72fab895,0.000000,0.000000,-1,1001_0_2,2025-10-14 13:37:25.513000+00:00,<NA>,NO_CHANGE,1001_0_2_20251014T133723_72fab895,<NA>,2025-10-14,20251014


In [68]:
events["next_odometer"] = events["odometer"].shift(-1)

In [69]:
events

,_id,agency_id,created_at,driver_id,extra_trip_id,latitude,longitude,odometer,pattern_id,received_at,stop_id,trigger_door,trip_id,vehicle_id,service_date,date_int,next_odometer
24179964,68f267fcabdd6324a4d45d29,43,2025-10-17 15:58:37+00:00,FFP18G,NaN,38.650478,-9.152249,192633808,NaN,2025-10-17 15:59:56.936000+00:00,1,NO_CHANGE,NaN,14,2025-10-17,20251017,192634048.0
24188019,68f2684be2dc246c99b9d9a5,43,2025-10-17 16:01:14+00:00,FFP18G,NaN,38.650208,-9.152725,192634048,NaN,2025-10-17 16:01:15.551000+00:00,1,OPENED,NaN,14,2025-10-17,20251017,192634048.0
24188513,68f26854df3e447bc9322fff,43,2025-10-17 16:01:23+00:00,FFP18G,NaN,38.650208,-9.152727,192634048,NaN,2025-10-17 16:01:24.665000+00:00,1,CLOSED,NaN,14,2025-10-17,20251017,192000944.0
220343,68ec949d50d52e23e1e3b42e,43,2025-10-13 05:56:44+00:00,ZBJHNF,NaN,38.673817,-9.153441,192000944,NaN,2025-10-13 05:56:45.788000+00:00,2,NO_CHANGE,NaN,14,2025-10-13,20251013,192044752.0
1215100,68ecc2443bad5529d99c3f23,43,2025-10-13 09:11:30+00:00,ZBJHNF,NaN,38.687992,-9.147430,192044752,NaN,2025-10-13 09:11:31.790000+00:00,2,NO_CHANGE,NaN,14,2025-10-13,20251013,192044800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7462865,68ee49937bdfa272f2964571,41,2025-10-14 13:01:06+00:00,5JE2NI,1001_0_1_20251014T130037_338f7cee,0.000000,0.000000,-1,1001_0_1,2025-10-14 13:01:07.965000+00:00,<NA>,NO_CHANGE,1001_0_1_20251014T130037_338f7cee,<NA>,2025-10-14,20251014,-1.0
7574522,68ee4eba1d39da2af2dcc93a,41,2025-10-14 13:23:05+00:00,5JE2NI,1001_0_1_20251014T131834_a9eee395,0.000000,0.000000,-1,1001_0_1,2025-10-14 13:23:05.994000+00:00,<NA>,NO_CHANGE,1001_0_1_20251014T131834_a9eee395,<NA>,2025-10-14,20251014,-1.0
7618721,68ee50d8abdd6324a4bb3646,41,2025-10-14 13:32:07+00:00,5JE2NI,1001_0_1_20251014T133055_3bc75041,0.000000,0.000000,-1,1001_0_1,2025-10-14 13:32:08.553000+00:00,<NA>,NO_CHANGE,1001_0_1_20251014T133055_3bc75041,<NA>,2025-10-14,20251014,-1.0
7644615,68ee52157bdfa272f2968d70,41,2025-10-14 13:37:24+00:00,5JE2NI,1001_0_2_20251014T133723_72fab895,0.000000,0.000000,-1,1001_0_2,2025-10-14 13:37:25.513000+00:00,<NA>,NO_CHANGE,1001_0_2_20251014T133723_72fab895,<NA>,2025-10-14,20251014,-1.0


In [ ]:
events["distance_m"] = (events["next_odometer"] - events["odometer"]).clip(lower=0)

In [73]:
routes = (events
      .dropna(subset=["vehicle_id","stop_id"])
      .groupby(["vehicle_id","stop_id"], as_index=False)
      .agg(
          stop_lon=("longitude", "first"),   # or "max"/"median" if you prefer
          stop_lat=("latitude", "first"),
          distance=("distance_m", "mean")
      ))

In [74]:
routes

,vehicle_id,stop_id,stop_lon,stop_lat,distance
0,14,1,-9.152249,38.650478,80.000000
1,14,2,-9.153441,38.673817,2626.800000
2,14,20028,-9.157301,38.681068,1906.126888
3,14,20029,-9.155580,38.682278,1600.848485
4,14,20030,-9.153488,38.683121,2148.338983
...,...,...,...,...,...
493081,13991,162004,-8.885792,38.529572,13140.000000
493082,13991,162005,-8.881873,38.529202,6941.059459
493083,13991,162006,-8.885794,38.529991,1895.777778
493084,13991,162007,-8.886128,38.528900,49056.000000


In [77]:
routes_valid = routes.merge(
    agg_validations,
    on=["stop_id", "vehicle_id"],          # add 'vehicle_id' here if validations has it
    how="inner"
)

# then join with stops (to enrich with official stop info)
routes_valid_stops = routes_valid.merge(
    stops,
    on="stop_id",
    how="left",
    suffixes=("", "_stop")
)


In [81]:
routes_valid_stops[['vehicle_id', 'stop_id', 'trip_id', 'stop_name', 'created_at']]

,vehicle_id,stop_id,trip_id,stop_name,created_at
0,14,20028,3005_0_3_1530_1559_0_ESC_DU,ALMADA (R BERNARDO FRANCISCO COSTA 73),2025-10-14 14:53:38+00:00
1,14,20031,3005_0_3_0930_0959_0_ESC_DU,ALMADA PRAÇA GIL VICENTE 4 (ESCOLA),2025-10-15 09:10:05+00:00
2,14,20031,3005_0_3_0930_0959_0_ESC_DU,ALMADA PRAÇA GIL VICENTE 4 (ESCOLA),2025-10-16 09:09:04+00:00
3,14,20031,3005_0_3_1130_1159_0_ESC_DU,ALMADA PRAÇA GIL VICENTE 4 (ESCOLA),2025-10-15 11:14:56+00:00
4,14,20031,3005_0_3_1200_1229_0_ESC_DU,ALMADA PRAÇA GIL VICENTE 4 (ESCOLA),2025-10-15 11:51:35+00:00
...,...,...,...,...,...
1231524,13991,162008,4549_0_1|1500|0750,Interface de Transportes de Setúbal (Terminal) P8,2025-10-15 07:13:55+00:00
1231525,13991,162008,4549_0_2|1500|0730,Interface de Transportes de Setúbal (Terminal) P8,2025-10-15 06:29:05+00:00
1231526,13991,162008,4562_0_1|1400|0915,Interface de Transportes de Setúbal (Terminal) P8,2025-10-14 08:21:22+00:00
1231527,13991,162008,4562_0_1|1400|1145,Interface de Transportes de Setúbal (Terminal) P8,2025-10-14 10:38:54+00:00


In [82]:
event_log = routes.merge(
    df_valid,
    on=["stop_id", "vehicle_id"],          # add 'vehicle_id' here if validations has it
    how="inner"
)

# then join with stops (to enrich with official stop info)
event_log_stops = event_log.merge(
    stops,
    on="stop_id",
    how="left",
    suffixes=("", "_stop")
)

In [87]:
routes_valid_stops.to_csv("data/out/routes.csv", index=True)

In [88]:
event_log_stops.to_csv("data/out/event_log.csv", index=True)

In [85]:
event_log_stops[['card_serial_number','vehicle_id', 'stop_id', 'trip_id', 'stop_name', 'created_at']]

,card_serial_number,vehicle_id,stop_id,trip_id,stop_name,created_at
0,T186D1,14,20028,3005_0_3_1530_1559_0_ESC_DU,ALMADA (R BERNARDO FRANCISCO COSTA 73),2025-10-14 14:53:38+00:00
1,4U61Y0,14,20031,3005_0_3_0930_0959_0_ESC_DU,ALMADA PRAÇA GIL VICENTE 4 (ESCOLA),2025-10-15 09:10:05+00:00
2,RLD5YQ,14,20031,3005_0_3_0930_0959_0_ESC_DU,ALMADA PRAÇA GIL VICENTE 4 (ESCOLA),2025-10-15 09:10:11+00:00
3,BHWV65,14,20031,3005_0_3_0930_0959_0_ESC_DU,ALMADA PRAÇA GIL VICENTE 4 (ESCOLA),2025-10-15 09:10:14+00:00
4,67GHH5,14,20031,3005_0_3_1130_1159_0_ESC_DU,ALMADA PRAÇA GIL VICENTE 4 (ESCOLA),2025-10-15 11:14:56+00:00
...,...,...,...,...,...,...
4411537,7AND4G,13991,162008,4562_0_1|1600|0845,Interface de Transportes de Setúbal (Terminal) P8,2025-10-16 07:43:34+00:00
4411538,MNBL0E,13991,162008,4562_0_1|1600|0845,Interface de Transportes de Setúbal (Terminal) P8,2025-10-16 07:43:38+00:00
4411539,A5PAH5,13991,162008,4562_0_1|1600|0845,Interface de Transportes de Setúbal (Terminal) P8,2025-10-16 07:43:42+00:00
4411540,CJLHW4,13991,162008,4562_0_1|1600|0845,Interface de Transportes de Setúbal (Terminal) P8,2025-10-16 07:43:48+00:00


#### Create GTFS for city2graph analysis

In [31]:
import pandas as pd
from pathlib import Path
import zipfile

def build_gtfs_from_raw(
    validations: pd.DataFrame,
    stops: pd.DataFrame,
    vehicles: pd.DataFrame,
    calendar_df: pd.DataFrame,
    agency_name: str = "Agency",
    agency_id: str = "AGENCY",
    tz: str = "Europe/Lisbon",
    route_type: int = 3,  # 3=bus (change if needed)
):
    """
    Expects:
      validations: columns like ['stop_id','trip_id','vehicle_id','line_id','created_at','service_date', ...]
      stops:      columns like ['stop_id','stop_lat','stop_lon'] (stop_name optional)
      vehicles:   optional (if you need operator-specific mapping)
      calendar_df: optional; if absent, we’ll synthesize a simple calendar
    Returns a dict of GTFS DataFrames.
    """

    v = validations.copy()
    s = stops.copy()

    # --- Timestamps & dates
    if "created_at" in v.columns:
        v["created_at"] = pd.to_datetime(v["created_at"], errors="coerce")
    if "service_date" in v.columns:
        v["service_date"] = pd.to_datetime(v["service_date"], errors="coerce").dt.date
    else:
        # derive service_date from created_at if not present
        v["service_date"] = v["created_at"].dt.date

    # ===== agency.txt =====
    agency = pd.DataFrame([{
        "agency_id": agency_id,
        "agency_name": agency_name,
        "agency_url": "https://example.org",
        "agency_timezone": tz,
        "agency_lang": "pt"  # change if needed
    }])

    # ===== routes.txt =====
    # Map your line_id to GTFS route_id/short_name
    if "line_id" not in v.columns:
        # fallback: derive a dummy route per trip prefix
        v["line_id"] = v["trip_id"].str.extract(r"^([^_]+)", expand=False).fillna("R0")

    routes = (
        v.dropna(subset=["line_id"])
         .drop_duplicates(subset=["line_id"])
         .rename(columns={"line_id": "route_id"})
         .assign(agency_id=agency_id,
                 route_short_name=lambda x: x["route_id"].astype(str),
                 route_long_name=pd.NA,
                 route_type=route_type)
         [["route_id","agency_id","route_short_name","route_long_name","route_type"]]
         .reset_index(drop=True)
    )

    # ===== trips.txt =====
    # Each distinct (trip_id, route_id) is a GTFS trip; we also attach service_id later
    trips_base = v[["trip_id","line_id","service_date"]].drop_duplicates()
    # Simple service_id: one per calendar day (you can map to your real calendar if you have one)
    trips_base["service_id"] = trips_base["service_date"]#.astype(str)
    trips = (
        trips_base.rename(columns={"line_id":"route_id"})
                  [["route_id","service_id","trip_id"]]
                  .drop_duplicates()
                  .reset_index(drop=True)
    )

    # ===== stops.txt =====
    # Make sure stops has the GTFS column names
    stops_cols_map = {
        "stop_lat":"stop_lat", "stop_lon":"stop_lon", "latitude":"stop_lat", "longitude":"stop_lon"
    }
    s = s.rename(columns={k:v for k,v in stops_cols_map.items() if k in s.columns and v not in s.columns})
    stops_gtfs = (
        s.dropna(subset=["stop_id","stop_lat","stop_lon"])
         .assign(stop_code=lambda x: x.get("stop_code", pd.Series([pd.NA]*len(x))),
                 stop_name=lambda x: x.get("stop_name", pd.Series([pd.NA]*len(x))),
                 wheelchair_boarding=0,
                 location_type=0)
         [["stop_id","stop_code","stop_name","stop_lat","stop_lon","wheelchair_boarding","location_type"]]
         .drop_duplicates("stop_id")
         .reset_index(drop=True)
    )

    # ===== stop_times.txt =====
    # Use the validations as observed stop times → one row per (trip_id, stop_id, created_at)
    st = (v.dropna(subset=["trip_id","stop_id","created_at"])
            .sort_values(["trip_id","created_at"])
            .assign(arrival_time=lambda x: x["created_at"].dt.strftime("%H:%M:%S"),
                    departure_time=lambda x: x["created_at"].dt.strftime("%H:%M:%S"))
         )

    # stop_sequence per trip
    st["stop_sequence"] = st.groupby("trip_id").cumcount() + 1

    # defaults
    st["stop_headsign"] = pd.NA
    st["pickup_type"] = 0
    st["drop_off_type"] = 0
    st["shape_dist_traveled"] = pd.NA
    st["timepoint"] = 0

    stop_times = st[[
        "trip_id","arrival_time","departure_time","stop_id","stop_sequence",
        "stop_headsign","pickup_type","drop_off_type","shape_dist_traveled","timepoint"
    ]].reset_index(drop=True)

    # ===== calendar.txt =====
    if calendar_df is not None:
        # Expect columns like: service_id, monday..sunday (0/1 or bool), start_date, end_date (YYYYMMDD)
        calendar = calendar_df.copy()
        # ensure types
        for col in ["start_date","end_date"]:
            if col in calendar.columns and pd.api.types.is_datetime64_any_dtype(calendar[col]):
                calendar[col] = calendar[col].dt.strftime("%Y%m%d")
    
    
    calendar['service_id'] = calendar['date_int'].astype(str)
    # Ensure trips reference valid service_ids
    #trips = trips.merge(calendar[["service_id"]].drop_duplicates(), on="service_id", how="inner")
    # Ensure trips reference valid routes
    trips = trips.merge(routes[["route_id"]], on="route_id", how="inner")

    gtfs = {
        "agency": agency,
        "routes": routes,
        "trips": trips,
        "stops": stops_gtfs,
        "stop_times": stop_times,
        "calendar": calendar
        # Optionally: calendar_dates, shapes, feed_info, frequencies
    }
    return gtfs

def write_gtfs_zip(gtfs_tables: dict, zip_path: Path):
    zip_path = Path(zip_path)
    with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for name, df in gtfs_tables.items():
            csv_name = f"{name}.txt"
            zf.writestr(csv_name, df.to_csv(index=False))

# -------------------------
# Example usage:
# gtfs = build_gtfs_from_raw(validations, stops, vehicles, calendar_df)
# write_gtfs_zip(gtfs, "my_feed_gtfs.zip")
# # then in city2graph:
# # gtfs_data = city2graph.load_gtfs("my_feed_gtfs.zip")



In [22]:
df_cal

,date_int,day_type,dia_tipo
0,20220101,3,DOM
1,20220102,3,DOM
2,20220103,1,DU
3,20220104,1,DU
4,20220105,1,DU
...,...,...,...
2917,20291227,1,DU
2918,20291228,1,DU
2919,20291229,2,SAB
2920,20291230,3,DOM


In [32]:
gtfs = build_gtfs_from_raw(df_valid, stops, df_event, df_cal)

In [35]:
write_gtfs_zip(gtfs, "data/out/gtfs.zip")

In [33]:
gtfs

{'agency':   agency_id agency_name           agency_url agency_timezone agency_lang
 0    AGENCY      Agency  https://example.org   Europe/Lisbon          pt,
 'routes':      route_id agency_id route_short_name route_long_name  route_type
 0        1709    AGENCY             1709            <NA>           3
 1        2769    AGENCY             2769            <NA>           3
 2        1218    AGENCY             1218            <NA>           3
 3        4600    AGENCY             4600            <NA>           3
 4        1107    AGENCY             1107            <NA>           3
 ..        ...       ...              ...             ...         ...
 685      2021    AGENCY             2021            <NA>           3
 686      4470    AGENCY             4470            <NA>           3
 687      4471    AGENCY             4471            <NA>           3
 688      4408    AGENCY             4408            <NA>           3
 689      4605    AGENCY             4605            <NA>    

In [34]:
gtfs['trips']

,route_id,service_id,trip_id
0,1709,2025-10-13,1709_0_1_0500_0529_0_1
1,2769,2025-10-13,2769_0_2|1|1|0455
2,1218,2025-10-13,1218_0_2_0430_0459_0_1
3,4600,2025-10-13,4600_0_2|1300|0500
4,1107,2025-10-13,1107_0_2_0430_0459_0_1
...,...,...,...
123345,2769,2025-10-20,2769_0_2|1|1|0455
123346,3105,2025-10-20,3105_0_1_0500_0529_0_ESC_DU
123347,2821,2025-10-20,2821_0_1|1|1|0450
123348,2730,2025-10-20,2730_0_2|1|1|0500
